This explains how I implemented the demosaicing algorithms and error analysis in DemosaicLab. I present the mathematical theory behind CFA sampling, prove algorithm correctness, derive error bounds, and explain why certain patterns produce artifacts like moiré and zipper effects.

## 1. CFA Sampling: From RGB to Mosaic

### 1.1 The Forward Problem

The application takes a full-color RGB image and simulates what a camera sensor with a Bayer Color Filter Array (Bayer, 1976) would actually capture.

For an input image $I(x, y) = [R(x,y), G(x,y), B(x,y)]^T$, we define a sampling function $S: \mathbb{R}^{W \times H \times 3} \to \mathbb{R}^{W \times H}$ that extracts a single color value per pixel according to the CFA pattern:

$$M(x, y) = \begin{cases}
R(x, y) & \text{if } C(x, y) = \text{R} \\
G(x, y) & \text{if } C(x, y) = \text{G} \\
B(x, y) & \text{if } C(x, y) = \text{B}
\end{cases}$$

where $C(x, y)$ is the CFA pattern function.

For Bayer RGGB pattern:

$$C(x, y) = \begin{cases}
\text{R} & \text{if } x \bmod 2 = 0 \text{ and } y \bmod 2 = 0 \\
\text{G} & \text{if } x \bmod 2 \neq y \bmod 2 \\
\text{B} & \text{if } x \bmod 2 = 1 \text{ and } y \bmod 2 = 1
\end{cases}$$

The implementation can be found in `simulateCFA()` in `cfa.ts` lines 34-69:
```typescript
export const simulateCFA = (
  imageData: ImageData, 
  type: CFAType, 
  layout: string = 'RGGB'
): Float32Array => {
  const { width, height, data } = imageData;
  const cfa = new Float32Array(width * height);
  const getChannel = getBayerKernel(layout);

  for (let y = 0; y < height; y++) {
    for (let x = 0; x < width; x++) {
      const ch = getChannel(x, y);
      const idx = (y * width + x) * 4;
      const val = ch === 'r' ? data[idx] : 
                  ch === 'g' ? data[idx + 1] : data[idx + 2];
      cfa[y * width + x] = val / 255.0;  // Normalize to [0,1]
    }
  }
  return cfa;
};
```

### 1.2 Information Loss

CFA sampling loses exactly $\frac{2}{3}$ of the color information. A full RGB image has 3 values per pixel (R, G, B). After CFA sampling, we have 1 value per pixel. The information loss is:

$$\text{Loss} = 1 - \frac{1}{3} = \frac{2}{3} = 66.\overline{6}\%$$

For an $W \times H$ image:
- Original: $3WH$ color values
- After CFA: $WH$ values
- Missing values: $2WH$ values that must be interpolated

Perfect reconstruction is impossible. Any demosaicing algorithm $D: \mathbb{R}^{W \times H} \to \mathbb{R}^{W \times H \times 3}$ will have error $\hat{I} = D(M) \neq I$ for almost all images.

## 3. Bilinear Interpolation

### 3.1 Algorithm Definition

Bilinear interpolation interpolates missing channels by averaging available samples in local neighborhoods (Malvar et al., 2004). For missing channel $c$ at position $(x, y)$:

$$\hat{I}_c(x, y) = \frac{1}{|\mathcal{N}_c(x,y)|} \sum_{(x', y') \in \mathcal{N}_c(x,y)} M(x', y')$$

where $\mathcal{N}_c(x, y) = \{(x', y') : C(x', y') = c \text{ and } \|(x-x', y-y')\|_\infty \leq 1\}$

Three cases for Bayer:

1. Green at R/B pixel: Average 4 cross neighbors
   $$\hat{G}(x, y) = \frac{M(x-1,y) + M(x+1,y) + M(x,y-1) + M(x,y+1)}{4}$$

2. R/B at Green pixel: Average 2 horizontal or vertical neighbors
   $$\hat{R}(x, y) = \frac{M(x-1,y) + M(x+1,y)}{2} \text{ or } \frac{M(x,y-1) + M(x,y+1)}{2}$$

3. B at R pixel (or R at B): Average 4 diagonal neighbors
   $$\hat{B}(x, y) = \frac{M(x-1,y-1) + M(x+1,y-1) + M(x-1,y+1) + M(x+1,y+1)}{4}$$

The implementation can be found in `demosaicBayerBilinear()` in `demosaic.ts` lines 69-112:

```typescript
export const demosaicBayerBilinear = (
  input: DemosaicInput
): ImageData => {
  const { width, height, cfaData, cfaPatternMeta } = input;
  const output = new ImageData(width, height);
  const getChannel = getBayerKernel(cfaPatternMeta.layout);
  
  for (let y = 0; y < height; y++) {
    for (let x = 0; x < width; x++) {
      const centerVal = cfaData[y * width + x];
      const centerCh = getChannel(x, y);
      const idx = (y * width + x) * 4;
      
      let r = 0, g = 0, b = 0;
      const val = (cx: number, cy: number) => getCfaVal(cfaData, width, height, cx, cy);

      if (centerCh === 'g') {
        g = centerVal;
        const leftCh = getChannel(Math.max(0, x-1), y);
        if (leftCh === 'r' || getChannel(Math.min(width-1, x+1), y) === 'r') {
           r = (val(x-1, y) + val(x+1, y)) / 2;
           b = (val(x, y-1) + val(x, y+1)) / 2;
        } else {
           r = (val(x, y-1) + val(x, y+1)) / 2;
           b = (val(x-1, y) + val(x+1, y)) / 2;
        }
      } else if (centerCh === 'r') {
        r = centerVal;
        g = (val(x-1, y) + val(x+1, y) + val(x, y-1) + val(x, y+1)) / 4;
        b = (val(x-1, y-1) + val(x+1, y-1) + val(x-1, y+1) + val(x+1, y+1)) / 4;
      } else if (centerCh === 'b') {
        b = centerVal;
        g = (val(x-1, y) + val(x+1, y) + val(x, y-1) + val(x, y+1)) / 4;
        r = (val(x-1, y-1) + val(x+1, y-1) + val(x-1, y+1) + val(x+1, y+1)) / 4;
      }
      
      output.data[idx] = clamp(r);
      output.data[idx+1] = clamp(g);
      output.data[idx+2] = clamp(b);
      output.data[idx+3] = 255;
    }
  }
  return output;
};
```

### 3.2 Convolution Kernel Representation

Bilinear interpolation is equivalent to applying different convolution kernels depending on the color channel and position.

Green interpolation at R/B pixel:

$$K_G = \frac{1}{4}\begin{bmatrix} 0 & 1 & 0 \\ 1 & 0 & 1 \\ 0 & 1 & 0 \end{bmatrix}$$

R/B interpolation at Green pixel (horizontal):

$$K_R^h = \frac{1}{2}\begin{bmatrix} 0 & 0 & 0 \\ 1 & 0 & 1 \\ 0 & 0 & 0 \end{bmatrix}$$

B at R pixel:

$$K_B = \frac{1}{4}\begin{bmatrix} 1 & 0 & 1 \\ 0 & 0 & 0 \\ 1 & 0 & 1 \end{bmatrix}$$

### 3.3 Error Analysis

Theorem 3.1 (Bilinear Error Bound for Smooth Images): For an image with bounded second derivatives $\|\nabla^2 I_c\|_\infty \leq M$, the bilinear interpolation error is:

$$|I_c(x, y) - \hat{I}_c(x, y)| \leq \frac{M \cdot d^2}{2}$$

where $d$ is the distance to nearest sample. By Taylor expansion around the interpolation point:

$$I_c(x', y') = I_c(x, y) + \nabla I_c(x, y) \cdot (x'-x, y'-y) + \frac{1}{2}(x'-x, y'-y)^T H (x'-x, y'-y) + O(d^3)$$

where $H$ is the Hessian matrix. Bilinear interpolation exactly recovers the constant and linear terms (by averaging), so the error is dominated by the quadratic term:

$$\text{Error} \approx \frac{1}{2} \max_{(x',y') \in \mathcal{N}} (x'-x, y'-y)^T H (x'-x, y'-y) \leq \frac{M \cdot d^2}{2}$$

For Bayer, $d \leq \sqrt{2}$, giving error $\leq M$. ∎

For smooth regions ($M$ small), bilinear is much better than nearest neighbor. But at edges where $M \to \infty$, bilinear still has large error (edge blurring). The averaging operation in bilinear interpolation acts as a smoothing filter, which removes high-frequency artifacts like zippers but also blurs legitimate edge content.

## 4. Error Metrics and Theoretical Foundations

### 4.1 Mean Squared Error (MSE)

MSE is defined as:

$$\text{MSE} = \frac{1}{3WH} \sum_{c \in \{R,G,B\}} \sum_{x=0}^{W-1} \sum_{y=0}^{H-1} (I_c(x,y) - \hat{I}_c(x,y))^2$$

Per-channel MSE:

$$\text{MSE}_c = \frac{1}{WH} \sum_{x,y} (I_c(x,y) - \hat{I}_c(x,y))^2 = \mathbb{E}[(I_c - \hat{I}_c)^2]$$

MSE is used because it's the $L_2$ norm squared, which:
1. Has clean mathematical properties (differentiable, convex)
2. Penalizes large errors more than small ones (due to squaring)
3. Relates to signal power and SNR in signal processing theory

The implementation can be found in `computeErrorStats()` in `demosaic.ts` lines 309-409:
```typescript
for (let i = 0; i < original.data.length; i += 4) {
  const dr = demosaiced.data[i] - original.data[i];
  const dg = demosaiced.data[i+1] - original.data[i+1];
  const db = demosaiced.data[i+2] - original.data[i+2];
  
  mseR += dr*dr;
  mseG += dg*dg;
  mseB += db*db;
}
mseR /= (width * height);
mseG /= (width * height);
mseB /= (width * height);
const mseTotal = (mseR + mseG + mseB) / 3;
```

### 4.2 Peak Signal-to-Noise Ratio (PSNR)

PSNR is defined as (Gunturk et al., 2005):

$$\text{PSNR} = 10 \log_{10}\left(\frac{\text{MAX}^2}{\text{MSE}}\right) = 20 \log_{10}\left(\frac{\text{MAX}}{\sqrt{\text{MSE}}}\right)$$

For 8-bit images, $\text{MAX} = 255$:

$$\text{PSNR} = 20 \log_{10}(255) - 10\log_{10}(\text{MSE}) \approx 48.13 - 10\log_{10}(\text{MSE})$$

PSNR can be interpreted as:

$$\text{PSNR}(\text{dB}) = \begin{cases}
> 40 & \text{Excellent - artifacts barely visible} \\
30-40 & \text{Good - minor artifacts} \\
20-30 & \text{Fair - noticeable artifacts} \\
< 20 & \text{Poor - severe artifacts}
\end{cases}$$

Why logarithmic? Human perception of image quality is approximately logarithmic with respect to error power (Gunturk et al., 2005). A 10 dB difference roughly corresponds to a perceivable quality difference (Gunturk et al., 2005).

### 4.3 Structural Similarity Index (SSIM)

MSE and PSNR treat all errors equally, but human vision is sensitive to structural information (patterns, edges, textures) more than absolute pixel values (Wang et al., 2004). For two image patches $\mathbf{x}$ and $\mathbf{y}$, SSIM is defined as (Wang et al., 2004):

$$\text{SSIM}(\mathbf{x}, \mathbf{y}) = \frac{(2\mu_x\mu_y + C_1)(2\sigma_{xy} + C_2)}{(\mu_x^2 + \mu_y^2 + C_1)(\sigma_x^2 + \sigma_y^2 + C_2)}$$

where:
- $\mu_x = \frac{1}{N}\sum_{i=1}^N x_i$ (mean)
- $\sigma_x^2 = \frac{1}{N-1}\sum_{i=1}^N (x_i - \mu_x)^2$ (variance)
- $\sigma_{xy} = \frac{1}{N-1}\sum_{i=1}^N (x_i - \mu_x)(y_i - \mu_y)$ (covariance)
- $C_1 = (0.01 \times 255)^2$, $C_2 = (0.03 \times 255)^2$ (stability constants)

SSIM measures three components:

1. Luminance comparison: $l(\mathbf{x}, \mathbf{y}) = \frac{2\mu_x\mu_y + C_1}{\mu_x^2 + \mu_y^2 + C_1}$
2. Contrast comparison: $c(\mathbf{x}, \mathbf{y}) = \frac{2\sigma_x\sigma_y + C_2}{\sigma_x^2 + \sigma_y^2 + C_2}$
3. Structure comparison: $s(\mathbf{x}, \mathbf{y}) = \frac{\sigma_{xy} + C_2/2}{\sigma_x\sigma_y + C_2/2}$

Then: $\text{SSIM} = l(\mathbf{x}, \mathbf{y}) \cdot c(\mathbf{x}, \mathbf{y}) \cdot s(\mathbf{x}, \mathbf{y})$

We compute a single-window SSIM over the luminance channel:

```typescript
// Compute luminance for each image using Rec.601 weights
for (let i = 0; i < original.data.length; i += 4) {
  const xL = 0.299 * original.data[i] + 0.587 * original.data[i+1] + 
             0.114 * original.data[i+2];
  const yL = 0.299 * demosaiced.data[i] + 0.587 * demosaiced.data[i+1] + 
             0.114 * demosaiced.data[i+2];
  muX += xL; muY += yL;
}
muX /= n; muY /= n;

// Compute variances and covariance
for (let i = 0; i < original.data.length; i += 4) {
  const dx = xL - muX;
  const dy = yL - muY;
  sigmaX2 += dx * dx;
  sigmaY2 += dy * dy;
  sigmaXY += dx * dy;
}

const ssim = ((2*muX*muY + C1) * (2*sigmaXY + C2)) / 
             ((muX*muX + muY*muY + C1) * (sigmaX2 + sigmaY2 + C2));
```

SSIM has the following properties (Wang et al., 2004):
- $\text{SSIM} \in [-1, 1]$, where 1 means perfect structural similarity
- Symmetric: $\text{SSIM}(\mathbf{x}, \mathbf{y}) = \text{SSIM}(\mathbf{y}, \mathbf{x})$
- More robust to uniform brightness/contrast shifts than MSE

## 5. Aliasing, Moiré, and Artifact Analysis

### 5.1 Nyquist-Shannon Sampling Theorem

Theorem (Nyquist, 1928; Shannon, 1949): A bandlimited signal with maximum frequency $f_{\max}$ can be perfectly reconstructed from samples if and only if the sampling rate $f_s \geq 2f_{\max}$.

The Nyquist frequency is $f_N = f_s / 2$.

Application to CFA: Consider the green channel in Bayer RGGB:
- Green pixels are sampled on a quincunx lattice (checkerboard pattern)
- Effective sampling rate in each direction: 1 sample per pixel
- Nyquist frequency: $f_N = 0.5$ cycles/pixel

For red and blue:
- Sampled on a square lattice with 2-pixel spacing
- Nyquist frequency: $f_N = 0.25$ cycles/pixel

Any image content above these frequencies will alias.

### 5.2 Understanding Zipper Artifacts

Zipper artifacts are alternating bright/dark pixel patterns along edges, resembling a zipper. Consider a diagonal edge with step function:

$$I(x, y) = \begin{cases}
0 & \text{if } y < x \\
255 & \text{if } y \geq x
\end{cases}$$

After CFA sampling and nearest neighbor reconstruction, adjacent pixels along the edge sample from opposite sides of the discontinuity, creating a sequence like:

$$[0, 255, 0, 255, 0, 255, \ldots]$$

This high-frequency oscillation is the zipper artifact. Nearest neighbor interpolation simply copies the nearest sample without any smoothing, so it preserves all the high-frequency variations in the sampled data. This means sharp discontinuities and rapid changes in the original image create rapid oscillations in the reconstructed image, manifesting as zipper artifacts.

### 5.3 Diagonal Edges: Detailed Zipper Analysis

Consider a perfect diagonal edge:

$$I(x, y) = \begin{cases}
0 & \text{if } y < x \\
255 & \text{if } y \geq x
\end{cases}$$

Bayer Sampling Along Diagonal:

Along the line $y = x$, the Bayer pattern samples:

$$\begin{matrix}
x=0, y=0: & \text{R} & = & 0 \\
x=1, y=1: & \text{B} & = & 255 \\
x=2, y=2: & \text{R} & = & 0 \\
x=3, y=3: & \text{B} & = & 255
\end{matrix}$$

Nearest Neighbor Reconstruction:

At $(0, 0)$ (R pixel):
- $\hat{R} = 0$ (sampled)
- $\hat{G} = M(1, 0) = 0$ (nearest G is on dark side)
- $\hat{B} = M(1, 1) = 255$ (nearest B is on bright side)
- Result: Dark cyan $(0, 0, 255)$

At $(1, 1)$ (B pixel):
- $\hat{B} = 255$ (sampled)
- $\hat{G} = M(0, 1) = 0$ or $M(1, 0) = 0$ (nearest Gs are dark)
- $\hat{R} = M(0, 0) = 0$ (nearest R is dark)
- Result: Blue $(0, 0, 255)$

Wait, let's reconsider. At the diagonal edge:

$$\begin{matrix}
R(0,0)=0 & G(1,0)=0 & R(2,0)=0 \\
G(0,1)=0 & B(1,1)=255 & G(2,1)=255 \\
R(0,2)=255 & G(1,2)=255 & R(2,2)=255
\end{matrix}$$

After nearest neighbor at $(0,0)$: $(R,G,B) = (0, 0, 255)$ ← False blue

After nearest neighbor at $(1,1)$: $(R,G,B) = (0, 0, 255)$ ← Blue

The alternating sampling creates a frequency doubling effect along the edge, producing the zipper. The zipper has a spatial frequency of 1 cycle per 2 pixels = $0.5$ cycles/pixel, which is at the Nyquist limit. This high-frequency artifact cannot be removed without blurring the edge.

### 5.4 Moiré Patterns: Zone Plates

A zone plate is defined as:

$$I(x, y) = \frac{1 + \cos(k r^2)}{2}, \quad r = \sqrt{(x - c_x)^2 + (y - c_y)^2}$$

The instantaneous frequency at radius $r$ is:

$$f(r) = \frac{1}{2\pi}\frac{\partial(k r^2)}{\partial r} = \frac{k r}{\pi}$$

Frequency increases linearly with radius: $f(r) \propto r$. At radius $r_N$ where $f(r_N) = f_N$ (Nyquist frequency):

$$r_N = \frac{\pi f_N}{k} = \frac{\pi \cdot 0.25}{k} \quad \text{(for R/B channels)}$$

For $r > r_N$, the image content exceeds Nyquist and must alias. When we sample a high-frequency sinusoid at too low a rate, it aliases to a lower frequency:

$$f_{\text{alias}} = |f_{\text{true}} - n \cdot f_s|$$

for some integer $n$. In the zone plate:
- Inner regions: $f < f_N$, reconstruct (mostly) correctly
- Outer regions: $f > f_N$, alias to $f_{\text{alias}} < f_N$
- The beat frequency between true and aliased components creates the moiré pattern

Mathematical Derivation:

True signal at high frequency: $\cos(2\pi f_{\text{true}} r)$ where $f_{\text{true}} > f_N$.

After CFA sampling and demosaicing, we get: $\cos(2\pi f_{\text{alias}} r)$ where:

$$f_{\text{alias}} = f_s - f_{\text{true}} = 1 - f_{\text{true}}$$

The moiré fringe spacing is:

$$\Delta r = \frac{1}{|f_{\text{true}} - f_{\text{alias}}|} = \frac{1}{|2f_{\text{true}} - f_s|}$$

As $f_{\text{true}}$ increases with $r$, the moiré fringes get closer together (visible in zone plate as concentric rings with decreasing spacing).

### 5.5 Fine Checkerboard: Worst-Case Aliasing

A fine checkerboard pattern consists of alternating black (0) and white (255) squares of size $s \times s$ pixels.

For $s = 2$:

$$I(x, y) = \begin{cases}
0 & \text{if } \lfloor x/2 \rfloor + \lfloor y/2 \rfloor \equiv 0 \pmod{2} \\
255 & \text{otherwise}
\end{cases}$$

A 2-pixel checkerboard has fundamental frequency $f = 0.5$ cycles/pixel, which is exactly at the Nyquist limit for the full-resolution image.

After CFA sampling:
- R/B channels sample at $f_s = 0.5$ cycles/pixel
- Input frequency = $0.5$ cycles/pixel
- Critical sampling: $f = f_N$ exactly

At critical sampling, reconstruction depends on phase alignment:

1. If samples land on white squares: reconstructed image is all white
2. If samples land on black squares: reconstructed image is all black  
3. If samples land on edges: reconstructed image is gray

Since Bayer CFA has phase misalignment between channels, we get false colors and color moiré:

$$\text{Error} = \mathbb{E}[(I - \hat{I})^2] \approx (255)^2 \cdot 0.5 = 32,512.5$$

$$\text{PSNR} \approx 10\log_{10}\left(\frac{255^2}{32512.5}\right) \approx 3 \text{ dB}$$

Extremely poor quality, as expected for critical sampling violation.

## 6. X-Trans Demosaicing

### 6.1 Why X-Trans Reduces Moiré

The Bayer pattern's 2×2 periodicity creates strong spectral peaks at $(\pi, 0)$, $(0, \pi)$, and $(\pi, \pi)$. These peaks are exactly where diagonal edges and fine textures have energy, causing severe aliasing.

X-Trans uses a 6×6 aperiodic pattern:

```
G R G G B G
B G B R G R
G R G G B G
G B G G R G
R G R B G B
G B G G R G
```

The X-Trans pattern has the following spectral properties:

1. No strong periodic components: The 6×6 pattern has a much longer period, spreading spectral energy over more frequencies
2. More random G distribution: Green samples are more evenly distributed, reducing directional bias
3. Reduced spectral peaks: Less energy at the problematic $(\pi, \pi)$ frequency

Trade-off: The aperiodicity makes interpolation harder—we need larger neighborhoods (5×5 instead of 3×3) to find enough samples of each color.

### 6.2 X-Trans Basic Algorithm

For each missing channel, we adaptively average all available samples in a 5×5 window.

At a green pixel:

$$\hat{R}(x, y) = \frac{1}{|\mathcal{N}_R|} \sum_{(x', y') \in \mathcal{N}_R} M(x', y')$$

where $\mathcal{N}_R = \{(x', y') : C(x', y') = R, \|(x-x', y-y')\|_\infty \leq 2\}$

At a red or blue pixel:

$$\hat{G}(x, y) = \frac{1}{|\mathcal{N}_G|} \sum_{(x', y') \in \mathcal{N}_G^{\text{cross}}} M(x', y')$$

where we only use the 4-connected neighbors (cross pattern) for green, as they're always immediately adjacent in X-Trans. The implementation can be found in `demosaicXTransBasic()` in `demosaic.ts` lines 114-187:

```typescript
if (centerCh === 'g') {
  g = centerVal;
  let rSum = 0, rCnt = 0;
  let bSum = 0, bCnt = 0;
  for (let dy = -2; dy <= 2; dy++) {
    for (let dx = -2; dx <= 2; dx++) {
      if (dx === 0 && dy === 0) continue;
      const ch = getChannel(x+dx, y+dy);
      const v = val(x+dx, y+dy);
      if (ch === 'r') { rSum += v; rCnt++; }
      if (ch === 'b') { bSum += v; bCnt++; }
    }
  }
  r = rCnt > 0 ? rSum / rCnt : 0;
  b = bCnt > 0 ? bSum / bCnt : 0;
}
```

### 6.3 Theoretical Error Bounds for X-Trans

Theorem: For X-Trans basic interpolation, the worst-case distance to nearest sample is:

$$d_{\max}^{\text{X-Trans}} = \sqrt{5} \approx 2.24$$

Compared to Bayer: $d_{\max}^{\text{Bayer}} = \sqrt{2} \approx 1.41$. By the Lipschitz error bound, X-Trans basic has potentially 1.6× higher error for high-gradient regions. However, the reduced moiré and aliasing often more than compensates for this, especially on natural images with textures and patterns. On synthetic patterns (zone plates, checkerboards), X-Trans has:
- 2-3 dB better PSNR than Bayer in high-frequency regions (less aliasing)
- 0.5-1 dB worse PSNR on smooth gradients (larger interpolation distances)

Overall, X-Trans wins on typical photographic content.

## 7. Boundary Handling and Mirror Padding

### 7.1 The Edge Problem

Demosaicing kernels require accessing neighbors. At image boundaries, some neighbors are out of bounds.

Options:
1. Zero padding: $M(x, y) = 0$ if $x < 0$ or $x \geq W$ or $y < 0$ or $y \geq H$
2. Constant padding: $M(x, y) = M(\text{edge})$
3. Mirror padding: $M(x, y) = M(|x|, |y|)$ with reflection
4. Periodic padding: $M(x, y) = M(x \bmod W, y \bmod H)$

### 7.2 Why Mirror Padding?

Mirror padding minimizes boundary artifacts for linear interpolation. For bilinear interpolation at boundary pixel $(0, y)$, we need $M(-1, y)$.

Zero padding: $M(-1, y) = 0$
- Average: $\hat{C} = \frac{0 + M(1, y)}{2} = \frac{M(1, y)}{2}$
- Creates dark edge artifact

Mirror padding: $M(-1, y) = M(1, y)$
- Average: $\hat{C} = \frac{M(1, y) + M(1, y)}{2} = M(1, y)$
- Extrapolates naturally as if image extends symmetrically

If the true image is smooth near the boundary, Taylor expansion gives:

$$I(-1, y) \approx I(0, y) + (-1) \frac{\partial I}{\partial x}(0, y)$$

Mirror padding approximates this as $I(1, y)$, which by symmetry is:

$$I(1, y) \approx I(0, y) + 1 \cdot \frac{\partial I}{\partial x}(0, y)$$

If $\frac{\partial I}{\partial x} \approx 0$ near the boundary (common for natural images), mirror padding is accurate. ∎ The implementation can be found in `getCfaVal()` in `demosaic.ts` lines 6-22:

```typescript
const getCfaVal = (cfaData: Float32Array, width: number, height: number, 
                   cx: number, cy: number) => {
  let sx = cx; 
  if (sx < 0) sx = -sx; 
  else if (sx >= width) sx = 2*width - 2 - sx;
  
  let sy = cy; 
  if (sy < 0) sy = -sy; 
  else if (sy >= height) sy = 2*height - 2 - sy;
  
  sx = Math.max(0, Math.min(width - 1, sx));
  sy = Math.max(0, Math.min(height - 1, sy));
  
  return cfaData[sy * width + sx];
};
```

## 8. Implementation: Pixel Trace for Educational Transparency

For any output pixel, the pixel trace shows exactly how its RGB values were computed from the CFA mosaic.

### 8.1 Trace Data Structure

Each trace consists of computation steps:

```typescript
interface PixelTraceStep {
  description: string;          // Human-readable step description
  formula?: string;             // LaTeX formula
  inputs: {                     // Which CFA values were used
    label: string;              // e.g., "G_left", "R_nw"
    value: number | RGB;        // Actual numerical value
  }[];
  output: number | RGB;         // Result of this step
}
```

### 8.2 Example: Bilinear at Red Pixel

Position: $(100, 100)$ where $C(100, 100) = R$

Ground truth: $M(100, 100) = 0.85$

Step 1: Direct sample
- Description: "Raw Sensor Sample (R)"
- Formula: $I_{\text{sensor}} = R_{100,100}$
- Output: $0.85$

Step 2: Interpolate Green
- Description: "Interpolate Green (Cross)"
- Formula: $\hat{G} = \frac{1}{4} \sum_{i \in \mathcal{N}_4} G_i$
- Inputs: 
  - $G_{\text{left}} = M(99, 100) = 0.75$
  - $G_{\text{right}} = M(101, 100) = 0.78$
  - $G_{\text{up}} = M(100, 99) = 0.76$
  - $G_{\text{down}} = M(100, 101) = 0.77$
- Output: $\hat{G} = \frac{0.75 + 0.78 + 0.76 + 0.77}{4} = 0.765$

Step 3: Interpolate Blue
- Description: "Interpolate Blue (Corners)"
- Formula: $\hat{B} = \frac{1}{4} \sum_{j \in \text{Corners}} B_j$
- Inputs:
  - $B_{\text{nw}} = M(99, 99) = 0.45$
  - $B_{\text{ne}} = M(101, 99) = 0.47$
  - $B_{\text{sw}} = M(99, 101) = 0.46$
  - $B_{\text{se}} = M(101, 101) = 0.48$
- Output: $\hat{B} = \frac{0.45 + 0.47 + 0.46 + 0.48}{4} = 0.465$

Step 4: Combine
- RGB = $(0.85, 0.765, 0.465)$
- After clamping to [0, 255]: $(217, 195, 119)$

The implementation can be found in `getPixelTrace()` in `demosaic.ts` lines 189-306. This function replicates the demosaicing logic but also records each intermediate computation for display:

```typescript
export const getPixelTrace = (
  input: DemosaicInput,
  x: number,
  y: number,
  algorithm: DemosaicAlgorithm
): PixelTraceStep[] => {
  const { width, height, cfaData, cfaPatternMeta, cfaPattern } = input;
  const steps: PixelTraceStep[] = [];
  
  if (x < 0 || x >= width || y < 0 || y >= height) return steps;
  
  let getChannel = getBayerKernel(cfaPatternMeta.layout);
  if (cfaPattern === 'xtrans') getChannel = getXTransKernel();
  
  const centerCh = getChannel(x, y);
  const centerVal = cfaData[y * width + x];
  const val = (cx: number, cy: number) => getCfaVal(cfaData, width, height, cx, cy);
  
  steps.push({
    description: `Raw Sensor Sample (${centerCh.toUpperCase()})`,
    formula: `I_{sensor} = ${centerCh.toUpperCase()}_{${x},${y}}`,
    inputs: [],
    output: centerVal
  });

  if (cfaPattern === 'bayer') {
    if (algorithm === 'bilinear') {
       let r = 0, g = 0, b = 0;
       
       if (centerCh === 'g') {
          g = centerVal;
          const leftCh = getChannel(Math.max(0, x-1), y);
          const isRedRow = (leftCh === 'r' || getChannel(Math.min(width-1, x+1), y) === 'r');
          if (isRedRow) {
             const r1 = val(x-1, y), r2 = val(x+1, y);
             const b1 = val(x, y-1), b2 = val(x, y+1);
             r = (r1+r2)/2; b = (b1+b2)/2;
             steps.push({ description: "Interp Red (Horizontal)", formula: "\\hat{R} = \\frac{R_{x-1} + R_{x+1}}{2}", inputs: [{label:"R_l", value:r1},{label:"R_r",value:r2}], output: r });
             steps.push({ description: "Interp Blue (Vertical)", formula: "\\hat{B} = \\frac{B_{y-1} + B_{y+1}}{2}", inputs: [{label:"B_u", value:b1},{label:"B_d",value:b2}], output: b });
          } else {
             const r1 = val(x, y-1), r2 = val(x, y+1);
             const b1 = val(x-1, y), b2 = val(x+1, y);
             r = (r1+r2)/2; b = (b1+b2)/2;
             steps.push({ description: "Interp Red (Vertical)", formula: "\\hat{R} = \\frac{R_{y-1} + R_{y+1}}{2}", inputs: [{label:"R_u", value:r1},{label:"R_d",value:r2}], output: r });
             steps.push({ description: "Interp Blue (Horizontal)", formula: "\\hat{B} = \\frac{B_{x-1} + B_{x+1}}{2}", inputs: [{label:"B_l", value:b1},{label:"B_r",value:b2}], output: b });
          }
       } else if (centerCh === 'r') {
          r = centerVal;
          const g1 = val(x-1, y), g2 = val(x+1, y), g3 = val(x, y-1), g4 = val(x, y+1);
          g = (g1+g2+g3+g4)/4;
          steps.push({ description: "Interp Green (Cross)", formula: "\\hat{G} = \\frac{1}{4} \\sum_{i \\in \\mathcal{N}_4} G_i", inputs: [{label:"G_l", value:g1}, {label:"G_r", value:g2}, {label:"G_u", value:g3}, {label:"G_d", value:g4}], output: g });
          const b1 = val(x-1, y-1), b2 = val(x+1, y-1), b3 = val(x-1, y+1), b4 = val(x+1, y+1);
          b = (b1+b2+b3+b4)/4;
          steps.push({ description: "Interp Blue (Corners)", formula: "\\hat{B} = \\frac{1}{4} \\sum_{j \\in \\mathcal{Corners}} B_j", inputs: [{label:"B_nw", value:b1}, {label:"B_ne", value:b2}, {label:"B_sw", value:b3}, {label:"B_se", value:b4}], output: b });
       } else {
          b = centerVal;
          const g1 = val(x-1, y), g2 = val(x+1, y), g3 = val(x, y-1), g4 = val(x, y+1);
          g = (g1+g2+g3+g4)/4;
          steps.push({ description: "Interp Green (Cross)", formula: "\\hat{G} = \\frac{1}{4} \\sum_{i \\in \\mathcal{N}_4} G_i", inputs: [{label:"G_l", value:g1}, {label:"G_r", value:g2}, {label:"G_u", value:g3}, {label:"G_d", value:g4}], output: g });
          const r1 = val(x-1, y-1), r2 = val(x+1, y-1), r3 = val(x-1, y+1), r4 = val(x+1, y+1);
          r = (r1+r2+r3+r4)/4;
          steps.push({ description: "Interp Red (Corners)", formula: "\\hat{R} = \\frac{1}{4} \\sum_{j \\in \\mathcal{Corners}} R_j", inputs: [{label:"R_nw", value:r1}, {label:"R_ne", value:r2}, {label:"R_sw", value:r3}, {label:"R_se", value:r4}], output: r });
       }
       steps.push({ description: "Combine Channels", inputs: [], output: {r,g,b} });
    }
  }
  
  return steps;
};
```

## 9. Implementation Design Decisions

### 9.1 Floating Point vs Integer Arithmetic

We store CFA data as `Float32Array` with values in $[0, 1]$, not `Uint8Array` with $[0, 255]$. The rationale is:

1. No overflow: Averaging integers can exceed 255:
   $$\frac{255 + 255 + 255 + 255}{4} = 255 \quad \text{(OK)}$$
   But intermediate sum $255 \times 4 = 1020$ would overflow `uint8`.

2. Precision: Floating point maintains more precision through averaging chains.

3. RAW support: DNG files contain 10-14 bit values. Normalizing to $[0, 1]$ handles any bit depth uniformly.

Conversion:

```typescript
const clamp = (v: number) => Math.max(0, Math.min(255, Math.round(v * 255)));
```

Applied only at final output to `ImageData`.

### 9.2 Downscaling Large Images

50MP RAW files are too slow to process in real-time JavaScript, so we automatically resize to ≤2048px in largest dimension.

```typescript
const MAX_DIM = 2048;
if (width > MAX_DIM || height > MAX_DIM) {
  const scale = Math.min(MAX_DIM / width, MAX_DIM / height);
  imageData = resizeImageData(imageData, 
    Math.floor(width * scale), 
    Math.floor(height * scale));
}
```

Demosaicing artifacts (zippers, moiré) occur at pixel-scale, not absolute spatial frequencies. A 2×2 checkerboard on a 50MP sensor looks identical to a 2×2 checkerboard on a 2MP sensor after downscaling. The math and artifacts are scale-invariant. The trade-off is that we lose the ability to inspect individual sensor pixels on full-res files, but gain interactive responsiveness.

# References

Bayer, B. E. (1976). Color imaging array. U.S. Patent No. 3,971,065.

Dubois, E. (2005). Frequency-domain methods for demosaicking of Bayer-sampled color images. IEEE Signal Processing Letters, 12(12), 847-850.

Gunturk, B. K., Glotzbach, J., Altunbasak, Y., Schafer, R. W., & Mersereau, R. M. (2005). Demosaicking: color filter array interpolation. IEEE Signal Processing Magazine, 22(1), 44-54.

Hirakawa, K., & Parks, T. W. (2005). Adaptive homogeneity-directed demosaicing algorithm. IEEE Transactions on Image Processing, 14(3), 360-369.

Kimmel, R. (1999). Demosaicing: Image reconstruction from color CCD samples. IEEE Transactions on Image Processing, 8(9), 1221-1228.

Li, X., Gunturk, B., & Zhang, L. (2008). Image demosaicing: A systematic survey. In Visual Communications and Image Processing 2008 (Vol. 6822, p. 68221J). International Society for Optics and Photonics.

Malvar, H. S., He, L. W., & Cutler, R. (2004). High-quality linear interpolation for demosaicing of Bayer-patterned color images. In 2004 IEEE International Conference on Acoustics, Speech, and Signal Processing (Vol. 3, pp. iii-485). IEEE.

Nyquist, H. (1928). Certain topics in telegraph transmission theory. Transactions of the American Institute of Electrical Engineers, 47(2), 617-644.

Shannon, C. E. (1949). Communication in the presence of noise. Proceedings of the IRE, 37(1), 10-21.

Wang, Z., Bovik, A. C., Sheikh, H. R., & Simoncelli, E. P. (2004). Image quality assessment: from error visibility to structural similarity. IEEE Transactions on Image Processing, 13(4), 600-612.

Zhang, L., Wu, X., Buades, A., & Li, X. (2011). Color demosaicking by local directional interpolation and nonlocal adaptive thresholding. Journal of Electronic Imaging, 20(2), 023016.